<a href="https://colab.research.google.com/github/FelixLeChat92/MermaidGen-Colab-Jupyter/blob/main/Mermaid_Gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Génération de diagrammes Mermaid

Ce notebook permet de générer des diagrammes Mermaid sous différents formats (SVG, PNG) et de les exporter. Il utilise les services en ligne [mermaid.live](https://mermaid.live/), [kroki.io](https://kroki.io/) et [mermaid.ink](https://mermaid.ink/) pour le rendu des diagrammes.

## Table des matières

- [Importation des librairies](#importation)
- [Définition du diagramme Mermaid](#diagramme-mermaid)  
- [Génération d'un lien d'édition mermaid.live](#lien-mermaid-live)
- [Exportation en SVG avec kroki.io](#export-svg-kroki)  
- [Exportation avec mermaid.ink](#export-mermaid-ink)
  - [Format SVG](#export-svg-mermaid-ink)
  - [Format PNG](#export-png-mermaid-ink)

## Importation des librairies <a name="importation"></a>

Les librairies suivantes sont utilisées dans ce notebook :

- `base64` : pour l'encodage et le décodage en base64
- `zlib` : pour la compression et décompression de données
- `json` : pour la manipulation de données JSON  
- `requests` : pour effectuer des requêtes HTTP
- `IPython.display` : pour afficher des images dans le notebook

In [1]:
import base64
import sys
import json
import zlib
from IPython.display import SVG, Image
import requests

## Définition du diagramme Mermaid <a name="diagramme-mermaid"></a>

Le code Mermaid du diagramme à générer est défini dans la variable `mermaid_code`.  
Modifiez cette variable pour changer le diagramme. Vous pouvez utiliser les différents types de diagramme mermaid disponible, et des caractères spéciaux gérés par l'utf-8

```mermaid
mindmap
  root((Racine))
    Branche 1
      Noeud 1
      Noeud 2
    Branche 2  
      Noeud 3
      Noeud 4
```

In [3]:
# Exemple d'utilisation
mermaid_code = '''
mindmap
  root((Médicaments de l'HTA))
    Diurétiques
      Thiazidiques
        Hydrochlorothiazide
        Indapamide
      Diurétiques de l'anse
'''

## Génération d'un lien d'édition mermaid.live <a name="lien-mermaid-live"></a>

La fonction `genPakoLink` génère un lien vers l'éditeur en ligne [mermaid.live](https://mermaid.live/) prérempli avec le code Mermaid défini précédemment.

Cliquez sur le lien généré pour ouvrir le diagramme dans mermaid.live et l'éditer interactivement.

In [ ]:
def js_btoa(data):
    return base64.b64encode(data)

def pako_deflate(data):
    compress = zlib.compressobj(9, zlib.DEFLATED, 15, 8, zlib.Z_DEFAULT_STRATEGY)
    compressed_data = compress.compress(data)
    compressed_data += compress.flush()
    return compressed_data

def genPakoLink(graphMarkdown: str):
    jGraph = {"code": graphMarkdown, "mermaid": {"theme": "standard"}}
    byteStr = json.dumps(jGraph).encode('utf-8')
    deflated = pako_deflate(byteStr)
    dEncode = js_btoa(deflated)
    link_code = dEncode.decode('ascii')
    return link_code

mermaid_link = genPakoLink(mermaid_code)
print("Lien mermaid.live : ")
print('http://mermaid.live/edit#pako:' + mermaid_link)


## Exportation en SVG avec kroki.io <a name="export-svg-kroki"></a>

La fonction `encode_mermaid_to_kroki_url` génère un lien vers une image SVG du diagramme en utilisant le service [kroki.io](https://kroki.io/).

Le code qui suit télécharge cette image SVG et l'enregistre dans le répertoire courant.

In [ ]:
def encode_mermaid_to_kroki_url(mermaid_diagram, compression_level=9):
    # Compresser le diagramme Mermaid avec zlib
    mermaid_compressed = zlib.compress(mermaid_diagram.encode('utf-8'), compression_level)

    # Encoder le diagramme compressé en base64 URL-safe
    mermaid_base64 = base64.urlsafe_b64encode(mermaid_compressed).decode('utf-8')

    # Construire l'URL Kroki complète
    kroki_url = f"https://kroki.io/mermaid/svg/{mermaid_base64}"

    return kroki_url

kroki_link = encode_mermaid_to_kroki_url(mermaid_code)
print(kroki_link)

string = kroki_link[-8:]
filename = string + ".svg"

response = requests.get(kroki_link)
if response.status_code == 200:
  # Enregistrer l'image SVG dans un fichier
  with open(filename, "wb") as file:
    file.write(response.content)
  print(f"L'image SVG a été enregistrée sous le nom '{filename}'")
else:
  print("Erreur lors du téléchargement de l'image SVG")
SVG(response.text)

## Exportation avec mermaid.ink <a name="export-mermaid-ink"></a>

### Format SVG <a name="export-svg-mermaid-ink"></a>

Cette section génère une image SVG du diagramme avec [mermaid.ink](https://mermaid.ink/), l'affiche dans le notebook et l'enregistre dans un fichier.

In [ ]:
def mermaid_ink_string(graph):
  graphbytes = graph.encode("utf-8")
  base64_bytes = base64.b64encode(graphbytes)
  base64_string = base64_bytes.decode("utf-8")
  string = base64_string
  return string

string = mermaid_ink_string(mermaid_code)
url_image_svg = "https://mermaid.ink/svg/" + string
print("lien vers l'image mermaid : " + url_image_svg)

filename = string[:8] + ".svg"

response = requests.get(url_image_svg)
if response.status_code == 200:
  # Enregistrer l'image SVG dans un fichier
  with open(filename, "wb") as file:
    file.write(response.content)
  print(f"L'image SVG a été enregistrée sous le nom '{filename}'")
else:
  print("Erreur lors du téléchargement de l'image SVG")
SVG(response.text)

##PNG - Qualité basse
### Format PNG <a name="export-png-mermaid-ink"></a>  

Similaire à la section précédente mais génère une image PNG (qualité plus faible).

In [ ]:
def mermaid_ink_string(graph):
  graphbytes = graph.encode("utf-8")
  base64_bytes = base64.b64encode(graphbytes)
  base64_string = base64_bytes.decode("utf-8")
  string = base64_string
  return string

string = mermaid_ink_string(mermaid_code)

url_image_png = "https://mermaid.ink/img/" + string
print("lien vers l'image mermaid : " + url_image_png)

filename = string[:8] + ".png"

response = requests.get(url_image_png)

display(Image(url=url_image_png))

response = requests.get(url_image_png)
if response.status_code == 200:
  # Enregistrer l'image SVG dans un fichier
  with open(filename, "wb") as file:
    file.write(response.content)
  print(f"L'image SVG a été enregistrée sous le nom '{filename}'")
else:
  print("Erreur lors du téléchargement de l'image PNG")

## Conclusion

Ce notebook facilite la génération et l'exportation de diagrammes Mermaid sous différents formats. Vous pouvez l'adapter à vos besoins en modifiant le code Mermaid dans la variable `mermaid_code`.

## Ressources

- Documentation Mermaid : [https://mermaid.js.org/](https://mermaid.js.org/)
- Éditeur en ligne mermaid.live : [https://mermaid.live/](https://mermaid.live/)
- Service de rendu d'images kroki.io : [https://kroki.io/](https://kroki.io/)
- Service mermaid.ink : [https://mermaid.ink/](https://mermaid.ink/)